# **Sentiment Analysis: Sparse Transfer Learning with the CLI**

In this example, you will fine-tune a 90% pruned BERT model onto some sentiment-analysis datasets using SparseML's CLI.

### **Sparse Transfer Learning Overview**

Sparse Transfer Learning is very similiar to typical fine-tuning you are used to when training models. However, with Sparse Transfer Learning, we start the training process from a pre-sparsified checkpoint and maintain the sparsity structure while the fine tuning occurs. At the end, you will have a sparse model trained on your dataset, ready to be deployed with DeepSparse for GPU-class performance on CPUs!

### **Pre-Sparsified BERT**
SparseZoo, Neural Magic's open source repository of pre-sparsified models, contains a 90% pruned version of BERT, which has been sparsified on the upstream Wikipedia and BookCorpus datasets with the
masked language modeling objective. [Check out the model card](https://sparsezoo.neuralmagic.com/models/nlp%2Fmasked_language_modeling%2Fobert-base%2Fpytorch%2Fhuggingface%2Fwikipedia_bookcorpus%2Fpruned90-none). We will use this model as the starting point for the transfer learning process.


***Let's dive in!***

## **Installation**

Install SparseML via `pip`.



In [1]:
%pip uninstall torch torchvision -y
%pip install sparseml[torch]

Found existing installation: torch 1.13.1+cu116
Uninstalling torch-1.13.1+cu116:
  Successfully uninstalled torch-1.13.1+cu116
Found existing installation: torchvision 0.14.1+cu116
Uninstalling torchvision-0.14.1+cu116:
  Successfully uninstalled torchvision-0.14.1+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.8/904.8 KB 48.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 91.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

If you are running on Google Colab, restart the runtime after this step.

In [1]:
!sparseml.transformers.text_classification --help

2023-02-27 03:10:00 sparseml.transformers WARNING  sparseml-transformers v4.23.1 installation not detected. Installing  sparseml-transformers v4.23.1 dependencies if transformers is already  installed in the environment, it will be overwritten. Set  environment variable NM_NO_AUTOINSTALL_TRANSFORMERS to disable
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 KB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

## **Sparse Transfer Learning with SST2**

SparseML's CLI enables you to kick-off training workflows with various utilities like dataset loading, checkpoint saving, 
metric reporting, and logging handled for you. All we have to do is pass a `model_name_or_path` (the starting checkpoint), a `task` (the GLUE task to train on), and a `recipe` (a YAML file specifying the sparsity related parameters) and we are up and running. The `recipes` is critical for instructing the training script how to modify the training process with sparsity related algorithms. For Sparse Transfer Learning, we will use a `recipe` that instructs SparseML to maintain sparsity during the training process and to apply quantization over the final few epochs. 

### **Run Transfer Learning**

For SST2, there is a pre-made transfer learning recipe available in [SparseZoo](https://sparsezoo.neuralmagic.com/models/nlp%2Fsentiment_analysis%2Fobert-base%2Fpytorch%2Fhuggingface%2Fsst2%2Fpruned90_quant-none). As such, we kick off transfer learning with the following:

In [ ]:
!sparseml.transformers.text_classification \
  --task_name sst2 \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/pruned90_quant-none \
  --distill_teacher zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/base-none \
  --output_dir sparse_quantized_bert-text_classification_sst2 \
  --max_seq_length 128 --per_device_train_batch_size 32 --per_device_eval_batch_size 32 --preprocessing_num_workers 6 \
  --do_train --do_eval --evaluation_strategy epoch --fp16 --save_strategy epoch --save_total_limit 1

Let's discuss the key arguments:
- `--task_name sst2` instructs SparseML to download and fine-tune onto the SST2 dataset. You can pass any GLUE task as the task name and SparseML automatically downloads the dataset from the Hugging Face hub.

- `--model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none` specifies the starting checkpoint for the fine tuning. Here, we passed a SparseZoo stub identifying the 90% pruned version of BERT trained with masked language modeling on the Wikipedia and BookCorpus datasets. SparseML downloads the checkpoint from the Zoo when the script starts.

- `--recipe zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/pruned90_quant-none` specifies the recipe to be applied by SparseML. Here, we passed a SparseZoo stub identifying the transfer learning recipe for the SST2 dataset. SparseML downloads the recipe from the Zoo when the script starts. See below for the details of what this recipe looks like.

- `--distill_teacher zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/base-none` is an optional argument that specifies a model to use for as a teacher to apply distillation during the training process. We passed a SparseZoo stub identifying a dense BERT model trained on SST2. SparseML downloads the teacher from the Zoo when the script starts.

The script downloads the starting checkpoint, the teacher model, and transfer learning recipe from SparseZoo as well as the SST2 
dataset and trains the model for 13 epochs, converging to ~92% accuracy on the validation set. The final model is quantized 
with 90% of weights pruned!

#### **Transfer Learning Recipe**

Here's what the transfer learning recipe for the SST2 dataset looks like.

The "Modifiers" are the important items that encode how SparseML should modify the training process for Sparse Transfer Learning:
- `ConstantPruningModifier` tells SparseML to pin weights at 0 over all epochs, maintaining the sparsity structure of the network
- `QuantizationModifier` tells SparseML to quanitze the weights with quantization aware training over the last 5 epochs
- `DistillationModifier` tells SparseML how to apply distillation to the model, including the layer and some hyperparameters

SparseML parses the modifiers and updates the training process to implement the algorithms and hyperparameters specified in the recipes.

In [6]:
from sparsezoo import Model
transfer_stub = "zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/pruned90_quant-none"
download_dir = "./transfer_recipe"
zoo_model = Model(transfer_stub, download_path=download_dir)
recipe_path = zoo_model.recipes.default.path
print(recipe_path)

./transfer_recipe/recipe/recipe_original.md


In [7]:
%cat ./transfer_recipe/recipe/recipe_original.md

<!--
Copyright (c) 2021 - present / Neuralmagic, Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
-->

---

version: 1.1.0

# General Variables
num_epochs: &num_epochs 13
init_lr: 1.5e-4
final_lr: 0

qat_start_epoch: &qat_start_epoch 8.0
observer_epoch: &observer_epoch 12.0
quantize_embeddings: &quantize_embeddings 1

distill_hardness: &distill_hardness 1.0
distill_temperature: &distill_temperature 2.0

weight_decay: 0.01

# Modifiers:

training_modifiers:
  - !EpochRangeModifier
      end_epoc

### **Export to ONNX**

Once you have trained your model, export to ONNX in order to deploy with DeepSparse. The artifacts of the training process 
are saved to your local filesystem. 

Run the following to convert your PyTorch checkpoint to ONNX:

In [ ]:
!sparseml.transformers.export_onnx \
  --model_path sparse_quantized_bert-text_classification_sst2 \
  --task text_classification

The script above creates a `deployment` folder in your local directory, which has all of the files needed for deployment with DeepSparse including the `model.onnx`, `config.json`, and `tokenizer.json` files.

## **Sparse Transfer Learning with a Custom Dataset**

Beyond the built-in GLUE tasks, we can also use a dataset from the Hugging Face Hub or pass via local files. Let's try an example of each for the sentiment analysis using [Rotten Tomatoes Dataset](https://huggingface.co/datasets/rotten_tomatoes), which containing 5,331 positive and 5,331 negative processed sentences.

For simplicity, we will perform the fine-tuning without distillation. Although the transfer learning recipe contains distillation
modifiers, by setting `--distill_teacher disable` we instruct SparseML to skip distillation.

### **Using a Hugging Face Dataset**

Let's walk through how to pass a Hugging Face dataset identifier to the CLI.

#### **Inspecting Rotten Tomatoes Dataset**

In [2]:
from datasets import load_dataset
from pprint import pprint

rotten_tomatoes = load_dataset("rotten_tomatoes")
print(rotten_tomatoes)
pprint(rotten_tomatoes["train"][0])

Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/921 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/488k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
{'label': 1,
 'text': 'the rock is destined to be the 21st century\'s new " conan " and '
         "that he's going to make a splash even greater than arnold "
         'schwarzenegger , jean-claud van damme or steven segal .'}


#### **Kick off Training**

To use this dataset with the CLI, we can replace the `--task_name sst2` argument with `--dataset_name rotten_tomatoes --input_column_names text --label_column_name label`. SparseML will then download the dataset from the Hugging Face hub and run training as before.

In [ ]:
!sparseml.transformers.text_classification \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/pruned90_quant-none \
  --recipe_args '{"num_epochs":12,"qat_start_epoch":7.0, "observer_epoch": 11.0}' \
  --distill_teacher disable \
  --dataset_name rotten_tomatoes --input_column_names "text" --label_column_name "label" \
  --output_dir sparse_quantized_bert-text_classification_rotten_tomatoes-hf_dataset --max_seq_length 128 --per_device_train_batch_size 32 \
  --per_device_eval_batch_size 32 --preprocessing_num_workers 6 --do_train --do_eval --evaluation_strategy epoch --fp16  \
  --save_strategy epoch --save_total_limit 1

You will notice that we used the same recipe as we did in the SST2 case (identified by the SparseZoo stub `zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/pruned90_quant-none`). Since the Rotten Tomatoes and SST2 tasks are similiar, we chose to start with the same hyperparameters as we used in SST2 training.


To update a recipe, you can download the YAML file from SparseZoo, make updates to the YAML directly, and pass the local path to SparseML. In this case, we used `--recipe_args '{"num_epochs":12,"qat_start_epoch":7.0, "observer_epoch": 11.0}'` to modify a recipe on the fly, updating to only run for 11 epochs.

### **Using Local CSV/JSON Files**

Let's walk through how to pass a CSV/JSON dataset to the CLI.

#### **Save Dataset as a CSV File**

For this example, we use Hugging Face `datasets` to create a CSV file for Rotten Tomatoes that can be passed to SparseML's CLI but you can use any framework you want to create the CSV.

In [4]:
from datasets import load_dataset
from pprint import pprint

rotten_tomatoes = load_dataset("rotten_tomatoes")
print(rotten_tomatoes)
pprint(rotten_tomatoes["train"][0])

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
{'label': 1,
 'text': 'the rock is destined to be the 21st century\'s new " conan " and '
         "that he's going to make a splash even greater than arnold "
         'schwarzenegger , jean-claud van damme or steven segal .'}


In [6]:
rotten_tomatoes["train"].to_csv("./rotten_tomatoes-train.csv")
rotten_tomatoes["validation"].to_csv("./rotten_tomatoes-validation.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

130634

We can see that the data is a CSV file with text and label as the columns:

In [8]:
!head ./rotten_tomatoes-train.csv --lines=10

,text,label
0,"the rock is destined to be the 21st century's new "" conan "" and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .",1
1,"the gorgeously elaborate continuation of "" the lord of the rings "" trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .",1
2,effective but too-tepid biopic,1
3,"if you sometimes like to go to the movies to have fun , wasabi is a good place to start .",1
4,"emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .",1
5,the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .,1
6,offers that rare combination of entertainment and education .,1
7,perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions 

In [7]:
!head ./rotten_tomatoes-validation.csv --lines=10

,text,label
0,compassionately explores the seemingly irreconcilable situation between conservative christian parents and their estranged gay and lesbian children .,1
1,the soundtrack alone is worth the price of admission .,1
2,rodriguez does a splendid job of racial profiling hollywood style--casting excellent latin actors of all ages--a trend long overdue .,1
3,"beneath the film's obvious determination to shock at any cost lies considerable skill and determination , backed by sheer nerve .",1
4,bielinsky is a filmmaker of impressive talent .,1
5,"so beautifully acted and directed , it's clear that washington most certainly has a new career ahead of him if he so chooses .",1
6,a visual spectacle full of stunning images and effects .,1
7,a gentle and engrossing character study .,1
8,"it's enough to watch huppert scheming , with her small , intelligent eyes as steady as any noir villain , and to enjoy the perfectly pitched web of tension that chabrol spins .",1


#### **Kick off Training**

To use the local files with the CLI, pass `--train_file ./rotten_tomatoes-train.csv --validation_file ./rotten_tomatoes-validation.csv  --input_column_names text --label_column_name label`.

Run the following:

In [ ]:
!sparseml.transformers.text_classification \
  --model_name_or_path zoo:nlp/masked_language_modeling/obert-base/pytorch/huggingface/wikipedia_bookcorpus/pruned90-none \
  --recipe zoo:nlp/sentiment_analysis/obert-base/pytorch/huggingface/sst2/pruned90_quant-none \
  --distill_teacher disable \
  --train_file ./rotten_tomatoes-train.csv --validation_file ./rotten_tomatoes-validation.csv  --input_column_names text --label_column_name label \
  --output_dir sparse_quantized_bert-text_classification_rotten_tomatoes-local_dataset --max_seq_length 128 --per_device_train_batch_size 32 \
  --per_device_eval_batch_size 32 --preprocessing_num_workers 6 --do_train --do_eval --evaluation_strategy epoch --fp16  \
  --save_strategy epoch --save_total_limit 1

2023-02-27 03:22:35.305202: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 03:22:36.332674: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 03:22:36.332817: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-02-27 03:22:36.332841: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT War